In [1]:
import pyspark
findspark.init('C:\Users\Marsh\spark-3.3.2-bin-hadoop3')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1259643663.py, line 2)

In [ ]:
df = spark.read.format("csv")
.option("inferSchema", "false")
    .option("header", "true")
    .option("sep", ",")
    .load("/data.csv")

df.write.format("delta").mode("append")
.save("delta/bronze_payments")

In [ ]:
df = spark.read.format("delta")
    .load("delta/bronze_payments")

from pyspark.sql.functions import sha2, concat_ws
df =df.withColumn("hash", sha2(concat_ws("||", *df.columns), 256))

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_payments")


In [ ]:
df = spark.table("silver_payments")

df = df.groupBy("date")

df.write.format("delta")
.mode("overwrite")
.saveAsTable("gold_payments")